In [ ]:
import pandas as pd
import numpy as np
from seak.lrt import pv_chi2mixture
# import seaborn as sns
# import matplotlib.pyplot as plt
import os

### Comparison of p-values from gene-specific LRT null distributions vs genome-wide (pooled) null distribution

In [ ]:
infile = snakemake.input.lrt_tsv

In [ ]:
results = pd.read_csv(infile, sep='\t')

In [ ]:
results.shape

In [ ]:
chi2mixparam = pd.read_csv(snakemake.input.chi2param, sep='\t', index_col=0)

In [ ]:
chi2mixparam

In [ ]:
# kern = ['linwb', 'linw', 'linwb_mrgLOF', 'linw_cLOF']
kern = snakemake.params.kern

In [ ]:
def calc_pval(k):
    return pv_chi2mixture(dof=chi2mixparam.loc['dof',k], mixture=chi2mixparam.loc['mixture',k], scale=chi2mixparam.loc['scale',k], stat=np.where(~results['scale_'+k].isna(), results['lrtstat_'+k], 0.))

In [ ]:
recalc_pval = pd.DataFrame({'pv_lrt_' + k: calc_pval(k) for k in kern},index=results.gene).reset_index()

In [ ]:
recalc_pval.head()

In [ ]:
pvcols = ['pv_lrt_' + k for k in kern]
res_merged = results[['gene'] + pvcols ].merge(recalc_pval, on='gene', suffixes=['_single','_pooled'])

In [ ]:
res_merged.set_index('gene', inplace=True)

In [ ]:
#plt.rcParams['figure.figsize'] = [5, 5]

In [ ]:
# plotting p-value of gene-specific LRT null distr. vs pooled LRT null distr.
# for i, _ in enumerate(kern):
#     sns.scatterplot(x='pv_lrt_'+kern[i]+'_single', y='pv_lrt_'+kern[i]+'_pooled', data=res_merged)
#     plt.title(kern[i])
#     plt.show()

In [ ]:
# plotting -log10(p-value) of gene-specific LRT null distr. vs pooled LRT null distr.
# for i, _ in enumerate(kern):
#     sns.scatterplot(x='pv_lrt_'+kern[i]+'_single', y='pv_lrt_'+kern[i]+'_pooled', data=-np.log10(res_merged))
#     plt.title(kern[i])
#     plt.show()

In [ ]:
cors = np.log10(res_merged).corr()

In [ ]:
# correlation of log10 p-values
cors

In [ ]:
res_merged['pheno'] = results['pheno'].iloc[0]

In [ ]:
outfile = os.path.dirname(snakemake.input.lrt_tsv) + '/pval_comparison_lrt_pooled_vs_single.tsv.gz'

In [ ]:
# export results
res_merged.to_csv(outfile, index=True, header=True, sep='\t')